In [3]:
import pandas as pd

In [12]:
path = '.\Downloads\Validation predictions_Yolo.csv'

In [32]:
val_compare = pd.read_csv(path, delimiter = ",")
val_compare.head()

,patientId,score_pred,x1_pred,y1_pred,width_pred,height_pred,x,y,width,height,Target
0,3a449d5f-28ae-4d1e-a613-7eeee7a04b62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,cbc35978-c3ec-4146-a5d3-8c89ce5a80c0,0.97,181.0,531.0,218.0,247.0,NaN,NaN,NaN,NaN,0
2,0d2bc8b2-2340-4deb-b29c-ae4da8fa5764,0.97,155.0,506.0,246.0,277.0,638.0,438.0,135.0,118.0,1
3,0d2bc8b2-2340-4deb-b29c-ae4da8fa5764,0.97,575.0,399.0,231.0,390.0,638.0,438.0,135.0,118.0,1
4,9f4cb263-83c4-494f-8b30-dfcb03efc791,1.00,203.0,486.0,218.0,221.0,NaN,NaN,NaN,NaN,0


In [34]:
val_compare.fillna(0, inplace = True)

In [35]:
val_compare.head()

,patientId,score_pred,x1_pred,y1_pred,width_pred,height_pred,x,y,width,height,Target
0,3a449d5f-28ae-4d1e-a613-7eeee7a04b62,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,cbc35978-c3ec-4146-a5d3-8c89ce5a80c0,0.97,181.0,531.0,218.0,247.0,0.0,0.0,0.0,0.0,0
2,0d2bc8b2-2340-4deb-b29c-ae4da8fa5764,0.97,155.0,506.0,246.0,277.0,638.0,438.0,135.0,118.0,1
3,0d2bc8b2-2340-4deb-b29c-ae4da8fa5764,0.97,575.0,399.0,231.0,390.0,638.0,438.0,135.0,118.0,1
4,9f4cb263-83c4-494f-8b30-dfcb03efc791,1.00,203.0,486.0,218.0,221.0,0.0,0.0,0.0,0.0,0


In [36]:
val_compare.dtypes

patientId       object
score_pred     float64
x1_pred        float64
y1_pred        float64
width_pred     float64
height_pred    float64
x              float64
y              float64
width          float64
height         float64
Target           int64
dtype: object

In [37]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 

In [38]:
results = confusion_matrix(val_compare['Target'], val_compare['score_pred'].round())  

In [30]:
# Defining function to find IoU scores
def IOU(x,y,width,height,x_pred,y_pred,width_pred,height_pred):
    w_intersection = min(x + width, x_pred + width_pred) - max(x, x_pred)
    h_intersection = min(y + height, y_pred + height_pred) - max(y, y_pred)
    if w_intersection <= 0 or h_intersection <= 0: # No overlap
        return 0
    I = w_intersection * h_intersection
    U = width * height + width_pred * height_pred - I # Union = Total Area - I
    return I / U

In [43]:
# Creating a column for IoU values
val_compare['IoU'] = val_compare.apply(lambda x: IOU(x['x'],x['y'],x['width'],x['height'],x['x1_pred'],x['y1_pred'],x['width_pred'],x['height_pred']), axis = 1)
val_compare.head()

,patientId,score_pred,x1_pred,y1_pred,width_pred,height_pred,x,y,width,height,Target,IoU
0,3a449d5f-28ae-4d1e-a613-7eeee7a04b62,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000
1,cbc35978-c3ec-4146-a5d3-8c89ce5a80c0,0.97,181.0,531.0,218.0,247.0,0.0,0.0,0.0,0.0,0,0.000000
2,0d2bc8b2-2340-4deb-b29c-ae4da8fa5764,0.97,155.0,506.0,246.0,277.0,638.0,438.0,135.0,118.0,1,0.000000
3,0d2bc8b2-2340-4deb-b29c-ae4da8fa5764,0.97,575.0,399.0,231.0,390.0,638.0,438.0,135.0,118.0,1,0.176823
4,9f4cb263-83c4-494f-8b30-dfcb03efc791,1.00,203.0,486.0,218.0,221.0,0.0,0.0,0.0,0.0,0,0.000000


In [44]:
print("Evaluation Metrics")
print(classification_report(val_compare['Target'], val_compare['score_pred'].round()))
print("confusion matrix")
print(results)
print("IoU Scores")
val_compare.groupby('Target')["IoU"].mean()
# accuracy: (tp + tn) / (p + n)
# precision tp / (tp + fp)
# recall: tp / (tp + fn)
# f1: 2 tp / (2 tp + fp + fn)
# Mean IoU: Mean value of intersection/ Union of the area and position of the bounding box ground truth and predictions

Evaluation Metrics
              precision    recall  f1-score   support

           0       0.93      0.22      0.36      1552
           1       0.31      0.96      0.47       563

   micro avg       0.42      0.42      0.42      2115
   macro avg       0.62      0.59      0.41      2115
weighted avg       0.77      0.42      0.39      2115

confusion matrix
[[ 349 1203]
 [  25  538]]
IoU Scores


Target
0    0.0000
1    0.2872
Name: IoU, dtype: float64